In [ ]:
import pandas as pd
import matplotlib.pyplot as mplt
#%matplotlib inline  
import nltk
import numpy as npy
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.probability import FreqDist
#conda install -c conda-forge textblob at Anaconda power prompt
#python -m textblob.download_corpora. This and the preceding line installs textblob
# run at the Anaconda Power prompt 
from textblob import TextBlob
output_file=open("amazon_reviews.csv",'w',encoding='utf-8')
input_file=open("Reviews.csv",'r',encoding='utf-8')
lines_to_read=10000
for i in range(lines_to_read):
 line=input_file.readline()
 output_file.write(line) 
output_file.close() 
input_file.close() 
#There are  many columns which are not useful for our sentiment analysis and
# hence it’s better to remove the columns which are not required.
# We may also create a separate dataframe containing only the requisite columns.
review_data=pd.read_csv("amazon_reviews.csv")
review_data.shape  #--- This is to know know the dimension of the dataframe
review_data.columns #--This is to know the column names of the dataframe
review_data['Score'] #---This is to see the review scores for different items
review_data['Score'].value_counts().plot(kind='bar') # This is to plot the bars
# for different ratings. The bar plot shows the highest rating of 5.
# for different ratings. The bar plot shows the highest rating of 5.
def pos_tag(text):
    try:
        return TextBlob(text).tags
    except:
        return None
review_data['pos'] = review_data['Summary'].apply(pos_tag)
review_data.to_csv("new_file.csv", index=False)
def get_adjectives(text):
    blob = TextBlob(text)
    return [ word for (word,tag) in blob.tags if tag.startswith("JJ")]


review_data['adjectives'] = review_data['Summary'].apply(get_adjectives)
review_data.columns
# Calculating Sentiment Values


review_data=review_data.assign(sentiment_value=review_data['HelpfulnessNumerator']/review_data['HelpfulnessDenominator'])


review_data['sentiment_value'] = review_data['sentiment_value'].fillna(0)
#review_data.replace(to_replace = npy.nan, value ='Negative')-- can also be 
#used for this purpose

review_data['sentiment_value']=review_data.sentiment_value.astype('int64')
review_data.columns

review_data.dtypes
review_data.loc[review_data.sentiment_value< 1, "sentiment_value"] = 0

review_data['sentiment_value'].replace(
    to_replace=[1],
   value='Positive',
   inplace=True
)
review_data['sentiment_value'].replace(
    to_replace=[0],
   value='Negative',
   inplace=True
)
# This is one way of interpretation
#for label, row in review_data.iterrows():
  # if review_data.loc[label,'sentiment_value'] == 'Positive':
   # print("Positively Reviewed")
 #  else:
   # print("Negatively Reviewed")   
 
# This is a better alternative
for ind in review_data.index:
    if review_data['sentiment_value'][ind]=='Positive':
        print(review_data['sentiment_value'][ind])
        print("The product with id:",review_data['ProductId'][ind],"has been  Positively reviewed")
    else:
        review_data['sentiment_value'][ind]
        print("The product with id:",review_data['ProductId'][ind],"has been Negatively reviewed")
#review_data.columns
review_data_part = review_data.iloc[:6137]
review_data_part["sentiment_value"].value_counts()

review_df = review_data[['adjectives','sentiment_value']]
#review_df
#The labels for this dataset are categorical. Machines understand only numeric
# data. So, we convert the categorical values to numeric using the factorize()
# method. This returns an array of numeric values and an Index of categories.

review_text=review_data.loc[:,'adjectives']
#review_text.head()
# Converting pandas array to numpy array
numpy_array = review_text.to_numpy()
# The numpy array is now saved in string format
npy.savetxt("reviews_last.txt", numpy_array, fmt = "%s")
# The string data is now read into text.
with open("reviews_last.txt",'r') as f:
     
     text = f.read()
f.close()

 #Passing the string text into word tokenizer
# for breaking the sentences
tokenized_text=sent_tokenize(text)
print("The tokenized Text is:",tokenized_text)
for i in tokenized_text:
  print (i)
import nltk
from nltk.corpus import stopwords
sw= set(stopwords.words('english'))
text1 = word_tokenize(text.lower())
print(text1)
stopwords = [x for x in text1 if x not in sw]
print(stopwords)
# Word tokenization irrespective of cases.
token = word_tokenize(text)
print("The tokenized Words are:",token)

#From the above output, we can see the text split into tokens. Words, comma, 
#punctuations are called tokens.

tokenizer = nltk.RegexpTokenizer(r"\w+") # This is to remove punctuation marks
new_words = tokenizer.tokenize(text)
print(new_words)
review_data_part=review_data_part.assign(final_adjectives=6137)
sentiment_label = review_data_part.sentiment_value.factorize()
sentiment_label
#The 0 here represents positive sentiment and the 1 represents negative
# sentiment.
#Now, we should transform our text data into something that our machine 
#learning model understands. Basically, we need to convert the text into an
# array of vector embeddings. Word embeddings are a beautiful way of
# representing the relationship between the words in the text. To do this,
# we first give each of the unique words a unique number and then replace 
#that word with the number assigned. Now,first, we retrieve all the text data
# from the adjective dataset as shown below.
#comment = review_df.adjectives.values
#print(comment)
comments = review_data_part.final_adjectives.values
print(comments)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
tokenizer = Tokenizer(num_words=1000)
#tokenizer.fit_on_texts(comments)
encoded_docs = tokenizer.texts_to_sequences(comments)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
vocab_size=1000
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(1000, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Train the sentiment analysis model
#Train the sentiment analysis model for 5 epochs on the whole dataset with 
#a batch size of 32 and a validation split of 20%.
# The following command is for training the model and it should be run only once 
# for a given dataset.for ind in df.index:
#The following training has been done. So, you need not run it.     
#history = model.fit(padded_sequence,sentiment_label[0],
                #  validation_split=0.2, epochs=5, batch_size=32)
def predict_sentiment(text):
    
    """tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(text).round().item())"""
  
    #if text in {'Good','good','great','Great','tasty','fresh','regular','GREAT','Delicious','Best','GOOD','smooth','Awesome','Natural','sensitive','healthy','convenient','wonderful','Excellent','Wonderful','unique','fantastic','favourite','delicious','strong','soft','Organic','Surprising','surprising','cutiest','Adorable','flavourful','fine','beautiful','fair'}:
    if sentiment_label[1][0] =='Positive' or text in {'bad','wrong','poor','expensive','gluten','DECEITFUL','Disappointed','artificial','addictive','Addictive','low','little','stale','crap','nasty','insane','hard','slight','outrageous','crazy','fattening','non','small','few'} and text not in {'perfect','able'}: 
        sentiment='Negative'
    else:
        sentiment='Positive'
    print("The expressed Sentiment is: ", sentiment)
    


for word in comments:
   print("For the word:",word) 
   predict_sentiment(word)
   
   
print(review_data.Score.value_counts())
Sentiment_count=review_data.groupby('ProductId').count()
print(Sentiment_count)
mplt.bar(Sentiment_count.index.values, Sentiment_count['Score'])
mplt.xlabel('Review Sentiments')
mplt.ylabel('Number of Review')
mplt.show()
# Generate word cloud
def plot_cloud(wordcloud):
    mplt.figure(figsize=(10, 10))
    mplt.imshow(wordcloud) 
    mplt.axis("off")
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='salmon', colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(text)
# Plot
plot_cloud(wordcloud)
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(text)
# Plot
plot_cloud(wordcloud)
#Finding frequency distinct in the text
 
# finding the frequency distinct in the tokens
# Importing FreqDist library from nltk and passing token into FreqDist

fdist = FreqDist(new_words)
print("The frequency of the tokenized Words:",fdist)
for i in fdist:
  print(i)
# To find the frequency of top 10 words
fdist1= fdist.most_common(10)
print("The frequency of the top ten words are:",fdist1)
for i in fdist1:
  print(i)

#Stemming
 
#Stemming usually refers to normalizing words into its base form or root form.

# Importing Porterstemmer from nltk library


pst = PorterStemmer()

# Importing LancasterStemmer from nltk

lst = LancasterStemmer()
#A list of words to be stemmed
word_list = new_words
print("{0:20}{1:20}{2:20}".format("Word","Porter Stemmer","lancaster Stemmer"))
for word in word_list:
    print("{0:20}{1:20}{2:20}".format(word,pst.stem(word),lst.stem(word)))

#Lancaster is more aggressive than Porter stemmer
#Lemmatization
#In simpler terms, it is the process of converting a word to its base form. The difference between stemming and lemmatization is, lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, often leading to incorrect meanings and spelling errors.

#For example, lemmatization would correctly identify the base form of ‘caring’ to ‘care’, whereas, stemming would cutoff the ‘ing’ part and convert it to car.

#Lemmatization can be implemented in python by using Wordnet Lemmatizer, Spacy Lemmatizer, TextBlob, Stanford CoreNLP

# Importing Lemmatizer library from nltk
print("The lemmatized  words in the tokenized words are as shown below:")
lemmatizer = WordNetLemmatizer() 
print("{0:20}{1:20}".format("Word","Lemmatized W
                        #Stop Words
 
#“Stop words” are the most common words in a language like “the”, “a”, “at”, “for”, “above”, “on”, “is”, “all”. These words do not provide any meaning and are usually removed from texts. We can remove these stop words using nltk library




#Part-of-speech tagging is used to assign parts of speech to each word of a given text (such as nouns, verbs, pronouns, adverbs, conjunction, adjectives, interjection) based on its definition and its context. There are many tools available for POS taggers and some of the widely used taggers are NLTK, Spacy, TextBlob, Standford CoreNLP, etc.

#Tokenize the text
tex = word_tokenize(text)
for token in tex:
   print(nltk.pos_tag([token]))

#Chunking
 
#Chunking means picking up individual pieces of information and grouping them into bigger pieces. In the context of NLP and text mining, chunking means a grouping of words or tokens into chunks.

token = word_tokenize(text)
tags = nltk.pos_tag(token)
reg = 'NP: {<DT>?<JJ>*<NN>}' 
a = nltk.RegexpParser(reg)
result = a.parse(tags)
print(result)ord"))
print("-----------------------------------------")
i=1
for word in text:
  print("{0:20}{1:20}".format(word,lemmatizer.lemmatize(word)))
  i=i+1
  if i>10:
    break
#Stop Words
 
#“Stop words” are the most common words in a language like “the”, “a”, “at”, “for”, “above”, “on”, “is”, “all”. These words do not provide any meaning and are usually removed from texts. We can remove these stop words using nltk library




#Part-of-speech tagging is used to assign parts of speech to each word of a given text (such as nouns, verbs, pronouns, adverbs, conjunction, adjectives, interjection) based on its definition and its context. There are many tools available for POS taggers and some of the widely used taggers are NLTK, Spacy, TextBlob, Standford CoreNLP, etc.

#Tokenize the text
tex = word_tokenize(text)
for token in tex:
   print(nltk.pos_tag([token]))

#Chunking
 
#Chunking means picking up individual pieces of information and grouping them into bigger pieces. In the context of NLP and text mining, chunking means a grouping of words or tokens into chunks.

token = word_tokenize(text)
tags = nltk.pos_tag(token)
reg = 'NP: {<DT>?<JJ>*<NN>}' 
a = nltk.RegexpParser(reg)
result = a.parse(tags)
print(result)